In [4]:
from enum import Enum
import re
from typing import Dict, List
from pydantic import BaseModel
from datetime import datetime

class classDetails(BaseModel):
    startTime: datetime
    endTime: datetime

    subject: str
    location: str

class Weekday(Enum):
    SEGUNDA_FEIRA = 0
    TERCA_FEIRA = 1
    QUARTA_FEIRA = 2
    QUINTA_FEIRA = 3
    SEXTA_FEIRA = 4
    SABADO = 5
    DOMINGO = 6

def rename_keys(dictionary: Dict[str, str]) -> Dict[Weekday, str]:
    return {Weekday[key.replace("-", "_").upper()]: value for key, value in dictionary.items() if key.replace("-", "_").upper() in Weekday.__members__}

def merge_events(dictionaryVariable: dict) -> dict:
    merged_dictionary = {}

    for day, events in dictionaryVariable.items():
        merged_events = []
        i = 0
        while i < len(events):
            current_event = events[i]
            # If there's a next event and it has the same subject as the current event
            if i + 1 < len(events) and current_event['subject'] == events[i + 1]['subject']:
                next_event = events[i + 1]
                # If the end time of the current event is the start time of the next event
                if current_event['time'].split(' - ')[1] == next_event['time'].split(' - ')[0]:
                    # Merge the events
                    merged_event = {
                        'time': current_event['time'].split(' - ')[0] + ' - ' + next_event['time'].split(' - ')[1],
                        'subject': current_event['subject'],
                        'location': current_event['location'] if current_event['location'] == next_event['location'] else current_event['location'] + ', ' + next_event['location']
                    }
                    current_event = merged_event  # Set the current event to the merged event
                    i += 1  # Skip the next event

            merged_events.append(current_event)
            i += 1

        merged_dictionary[day] = merged_events

    return merged_dictionary

def preliminary_text_to_dict(scheduleText: str) -> Dict[str, str]:  # just use the function to be more clear the output
    schedule = {}
    weekdays = ['SEGUNDA-FEIRA', 'TERÇA-FEIRA', 'QUARTA-FEIRA', 'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SABADO', 'DOMINGO']
    lines = scheduleText.splitlines()
    current_day = None

    # create dictionary with keys as weekdays and values as a list with every index as one class time entry
    #{'SEGUNDA-FEIRA': ['10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula', '10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula'], 'QUINTA-FEIRA': ['10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula', '10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula'], 'SEXTA-FEIRA': ['17:10 - 18:00    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula', '18:00 - 18:50    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula']}
    for line in lines:
        if line in weekdays:
            current_day = line
            schedule[current_day] = []
        elif current_day is not None:
            schedule[current_day] += line.strip().split('\n')

    
    # the dictionary schedule will be updated in a way that the keys will continue to be the days of the week but
    # values will be a list of dictionaries, every dictionary being a class time entry
    # and this dictionary will have keys: time,subject,location
    for key,value in schedule.items():
        lst = []
        for entry in value:
            parts = re.split('\t', entry)
            print(parts)
            time = parts[0]
            subject = parts[1].split('-')[-1]
            location = parts[2].split('-')[0]
            lst.append({'time':time,
                        'subject': subject,
                        'location' :location})
        schedule[key]= lst
    return merge_events(schedule)
        

def parse_schedule_text(scheduleText: str) -> Dict[Weekday, List[classDetails]]:
    
    preliminary_dict = preliminary_text_to_dict(scheduleText)
    scheduleDict = rename_keys(preliminary_dict)
    
    for day, classes in scheduleDict.items():
      for index, classDetails in enumerate(classes):
        startTimeString = classDetails.get('time').split('-')[0].strip()
        endTimeString = classDetails.get('time').split('-')[-1].strip()

        startTime = datetime.strptime(startTimeString, '%H:%M')
        endTime = datetime.strptime(endTimeString, '%H:%M')
        scheduleDict[day][index]['startTime'] = startTime
        scheduleDict[day][index]['endTime'] = endTime
        del classDetails['time']
    
    return scheduleDict
    

In [5]:
text2 = """SEGUNDA-FEIRA
08:00 - 08:50	22000174 - T1 - ENGENHARIA DE SOFTWARE II	[ANG] 235 - Sala de Aula
08:50 - 09:40	22000174 - T1 - ENGENHARIA DE SOFTWARE II	[ANG] 235 - Sala de Aula
15:10 - 16:00	22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL	[ANG] 220 - Sala de Aula
16:00 - 16:50	22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL	[ANG] 220 - Sala de Aula
17:10 - 18:00	22000346 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO V	[ANG] 330 - Laboratório Computação 2
18:00 - 18:50	22000346 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO V	[ANG] 330 - Laboratório Computação 2
TERÇA-FEIRA
08:00 - 08:50	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:00 - 08:50	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:50 - 09:40	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:50 - 09:40	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
15:10 - 16:00	22000187 - T1 - COMPUTAÇÃO GRÁFICA	[ANG] 330 - Laboratório Computação 2
16:00 - 16:50	22000187 - T1 - COMPUTAÇÃO GRÁFICA	[ANG] 330 - Laboratório Computação 2
QUARTA-FEIRA
10:00 - 10:50	22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL	[ANG] 336 - Laboratório Computação 5
10:50 - 11:40	22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL	[ANG] 336 - Laboratório Computação 5
QUINTA-FEIRA
08:00 - 08:50	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:00 - 08:50	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:50 - 09:40	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
08:50 - 09:40	22000186 - T1 - COMPUTAÇÃO E SOCIEDADE	[ANG] 342 - Sala de aula
13:30 - 14:20	22000187 - T1 - COMPUTAÇÃO GRÁFICA	[ANG] 330 - Laboratório Computação 2
14:20 - 15:10	22000187 - T1 - COMPUTAÇÃO GRÁFICA	[ANG] 330 - Laboratório Computação 2
15:10 - 16:00	22000351 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO X	[ANG] 220 - Sala de Aula
16:00 - 16:50	22000351 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO X	[ANG] 220 - Sala de Aula
17:10 - 18:00	22000351 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO X	[ANG] 330 - Laboratório Computação 2
18:00 - 18:50	22000351 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO X	[ANG] 330 - Laboratório Computação 2
SEXTA-FEIRA
08:00 - 08:50	22000272 - T1 - INTROD PROCESSAMENTO PARALELO E DISTRIBUÍDO 	[ANG] 314 - Sala de aula
08:50 - 09:40	22000272 - T1 - INTROD PROCESSAMENTO PARALELO E DISTRIBUÍDO 	[ANG] 314 - Sala de aula
10:00 - 10:50	22000272 - T1 - INTROD PROCESSAMENTO PARALELO E DISTRIBUÍDO 	[ANG] 314 - Sala de aula
10:50 - 11:40	22000272 - T1 - INTROD PROCESSAMENTO PARALELO E DISTRIBUÍDO 	[ANG] 314 - Sala de aula
15:10 - 16:00	22000174 - T1 - ENGENHARIA DE SOFTWARE II	[ANG] 314 - Sala de aula
16:00 - 16:50	22000174 - T1 - ENGENHARIA DE SOFTWARE II	[ANG] 314 - Sala de aula
17:10 - 18:00	22000305 - M1 - TRABALHO DE CONCLUSÃO DE CURSO I	[ANG] 235 - Sala de Aula
18:00 - 18:50	22000305 - M1 - TRABALHO DE CONCLUSÃO DE CURSO I	[ANG] 235 - Sala de Aula"""


preliminary_text_to_dict(text2)


['08:00 - 08:50', '22000174 - T1 - ENGENHARIA DE SOFTWARE II', '[ANG] 235 - Sala de Aula']
['08:50 - 09:40', '22000174 - T1 - ENGENHARIA DE SOFTWARE II', '[ANG] 235 - Sala de Aula']
['15:10 - 16:00', '22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL', '[ANG] 220 - Sala de Aula']
['16:00 - 16:50', '22000268 - T2 - CÁLCULO NUMÉRICO COMPUTACIONAL', '[ANG] 220 - Sala de Aula']
['17:10 - 18:00', '22000346 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO V', '[ANG] 330 - Laboratório Computação 2']
['18:00 - 18:50', '22000346 - T1 - TÓPICOS ESPECIAIS EM COMPUTAÇÃO V', '[ANG] 330 - Laboratório Computação 2']
['08:00 - 08:50', '22000186 - T1 - COMPUTAÇÃO E SOCIEDADE', '[ANG] 342 - Sala de aula']
['08:00 - 08:50', '22000186 - T1 - COMPUTAÇÃO E SOCIEDADE', '[ANG] 342 - Sala de aula']
['08:50 - 09:40', '22000186 - T1 - COMPUTAÇÃO E SOCIEDADE', '[ANG] 342 - Sala de aula']
['08:50 - 09:40', '22000186 - T1 - COMPUTAÇÃO E SOCIEDADE', '[ANG] 342 - Sala de aula']
['15:10 - 16:00', '22000187 - T1 - COMPUTAÇÃO GRÁFIC

{'SEGUNDA-FEIRA': [{'time': '08:00 - 09:40',
   'subject': ' ENGENHARIA DE SOFTWARE II',
   'location': '[ANG] 235 '},
  {'time': '15:10 - 16:50',
   'subject': ' CÁLCULO NUMÉRICO COMPUTACIONAL',
   'location': '[ANG] 220 '},
  {'time': '17:10 - 18:50',
   'subject': ' TÓPICOS ESPECIAIS EM COMPUTAÇÃO V',
   'location': '[ANG] 330 '}],
 'TERÇA-FEIRA': [{'time': '08:00 - 08:50',
   'subject': ' COMPUTAÇÃO E SOCIEDADE',
   'location': '[ANG] 342 '},
  {'time': '08:00 - 09:40',
   'subject': ' COMPUTAÇÃO E SOCIEDADE',
   'location': '[ANG] 342 '},
  {'time': '08:50 - 09:40',
   'subject': ' COMPUTAÇÃO E SOCIEDADE',
   'location': '[ANG] 342 '},
  {'time': '15:10 - 16:50',
   'subject': ' COMPUTAÇÃO GRÁFICA',
   'location': '[ANG] 330 '}],
 'QUARTA-FEIRA': [{'time': '10:00 - 11:40',
   'subject': ' CÁLCULO NUMÉRICO COMPUTACIONAL',
   'location': '[ANG] 336 '}],
 'QUINTA-FEIRA': [{'time': '08:00 - 08:50',
   'subject': ' COMPUTAÇÃO E SOCIEDADE',
   'location': '[ANG] 342 '},
  {'time': '08:0